In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2


from pathlib import Path

import gc
import time

from utils import *

seed=42
PATH = Path('data')
list(PATH.iterdir())

[WindowsPath('data/blending'),
 WindowsPath('data/ipapp.pkl'),
 WindowsPath('data/submission'),
 WindowsPath('data/test.csv'),
 WindowsPath('data/test_nextclick_FE.feather'),
 WindowsPath('data/train_day8_3to16_nextclick_FE.feather'),
 WindowsPath('data/train_day9_3to16_nextclick_FE.feather'),
 WindowsPath('data/validation')]

# Simple average

In [10]:
nn_pred = pd.read_csv(PATH/'submission'/'imbalanced_data.csv') # .9728
fm_pred =  pd.read_csv(PATH/'submission'/'wordbatch_fm_ftrl.csv')# .9769
xgb_pred = pd.read_csv(PATH/'submission'/'XGB_FE_day89_ratio2_10mil_moreite.csv') # .9777
lgb_pred = pd.read_csv(PATH/'submission'/'sub-it200102.csv') # .9811?

In [11]:
# final_pred = xgb_pred.is_attributed*0.4 + nn_pred.is_attributed*0.3+fm_pred.is_attributed*0.3
final_pred = lgb_pred.is_attributed*0.5 + nn_pred.is_attributed*0.1+fm_pred.is_attributed*0.2 + xgb_pred.is_attributed*0.2

In [3]:
# final_pred = xgb_pred.is_attributed*0.25 + nn_pred.is_attributed*0.25+fm_pred.is_attributed*0.25 + lgb_pred.is_attributed*0.25


In [12]:
sub = xgb_pred.copy()
sub.is_attributed = final_pred
# sub.to_csv(PATH/'submission'/'blending_4_models_even.csv',index=False)
sub.to_csv(PATH/'submission'/'blending_bestlgb_5122.csv',index=False)

In [13]:
sub.tail()

,click_id,is_attributed
18790464,18790464,0.108887
18790465,18790465,0.000395
18790466,18790467,0.019053
18790467,18790466,0.027378
18790468,18790468,0.001248


# Use chunksize in read_csv

In [34]:
temp_df = pd.read_csv(PATH/'test.csv',chunksize=3000000)

In [36]:
for chunk in temp_df:
    print(chunk.head())

   click_id      ip  app  device  os  channel           click_time
0         0    5744    9       1   3      107  2017-11-10 04:00:00
1         1  119901    9       1   3      466  2017-11-10 04:00:00
2         2   72287   21       1  19      128  2017-11-10 04:00:00
3         3   78477   15       1  13      111  2017-11-10 04:00:00
4         4  123080   12       1  13      328  2017-11-10 04:00:00
         click_id     ip  app  device  os  channel           click_time
3000000   3000000  94836   15       1  19      259  2017-11-10 04:53:29
3000001   3000001   5314   18       1  13      379  2017-11-10 04:53:29
3000002   3000002  52186   25       1  13      259  2017-11-10 04:53:29
3000003   3000003   2821   20       1  13      478  2017-11-10 04:53:29
3000004   3000004   5314   18       1  19      107  2017-11-10 04:53:29
         click_id     ip  app  device  os  channel           click_time
6000000   6000001  55910    2       1  22      364  2017-11-10 05:55:22
6000001   6000000  882

# numpy memmap: Mapping new features (in pickles or feather) in existing memmap

Read this first: https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.memmap.html

numpy.memmap: Create a memory-map to an array stored in a binary file on disk.

Memory-mapped files are used for accessing small segments of large files on disk, without reading the entire file into memory


The memmap object can be used anywhere an ndarray is accepted. Given a memmap fp, isinstance(fp, numpy.ndarray) returns True.

In [3]:
temp_df = pd.read_csv(PATH/'test.csv')

In [53]:
temp_df[['ip','app']].to_pickle(PATH/'ipapp.pkl')

In [54]:
temp_df.drop(['ip','app'],axis=1,inplace=True)
temp_df.to_csv('test2.csv',index=False)

In [55]:
temp_df.head()

,click_id,device,os,channel,click_time
0,0,1,3,107,2017-11-10 04:00:00
1,1,1,3,466,2017-11-10 04:00:00
2,2,1,19,128,2017-11-10 04:00:00
3,3,1,13,111,2017-11-10 04:00:00
4,4,1,13,328,2017-11-10 04:00:00


## start

In [4]:
split_train = 10000000     # border between train/test in pickle (length of our train)
start_skip = 0   # if we want strip our train
n_attrs = 7
pickle_list = ['ipapp']   # list of pickled attrs
del_cols = []  # attrs to be deleted in final train

In [5]:
si = 0   
mmap = np.memmap(r'./data/mmap_train.mmp', dtype='float32', mode='w+', shape=(split_train, n_attrs))

In [6]:
def load_attrs(fname, data_dir='./data/'):
    fname = data_dir + fname + '.pkl'
    print('loading {}... '.format(fname))
    return pd.read_pickle(fname)

In [7]:
columns = []
for pkl in pickle_list:  
    temp = load_attrs(pkl)

    columns +=[x for x in temp.columns if x not in del_cols]  
    
    cols_idx = [temp.columns.tolist().index(x) for x in temp.columns if x not in del_cols]  
    temp = temp.iloc[start_skip:split_train, cols_idx]  

    ei = temp.values.shape[1]  
    
    # Assign value to memmap
    mmap[:, si:si+ei] = temp.values  
    si += ei  

    del temp  
    gc.collect()

loading ./data/ipapp.pkl... 
[0, 1]


At this point, only cols [0,1] of mmap is filled. Memmap should be filled with existing features before

In [9]:
mmap.flush()
del mmap
gc.collect()

0

In [10]:
val_size=50000

In [11]:
mmap = np.memmap(r'./data/mmap_train.mmp', dtype='float32', mode='r', shape=(split_train, n_attrs))

# Only write data we wants to memory. 
train = np.array(mmap[start_skip:-val_size])
val = np.array(mmap[-val_size:])

train and val are numpy arrays now. Note that mmap should have col 2 -> 6 filled

In [19]:
print(_train[:,0:2])

[[5.74400e+03 9.00000e+00]
 [1.19901e+05 9.00000e+00]
 [7.22870e+04 2.10000e+01]
 ...
 [1.24611e+05 1.20000e+01]
 [2.79480e+04 1.20000e+01]
 [5.31400e+03 2.80000e+01]]


In [21]:
print(_train[:,3:])

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 ...
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [ ]:
y_train = _train[:, columns.index('is_attributed')]
y_val = _val[:, columns.index('is_attributed')]

In [ ]:
# Example of using what is read from memmap (ndarray) in LGB

use_columns = [...]
use_columns_idxs = [columns.index(x) for x in use_columns]

d_train = train[:, use_columns_idxs]
xgtrain = lgb.Dataset(d_train, label=y_train, **dataset_params)
d_val = val[:, use_columns_idxs]
xgvalid = lgb.Dataset(d_val, label=y_val, reference=xgtrain, **dataset_params)